In [ ]:
# arr = ds_lazy['LC09_L1TP_141047_20250519_20250519_02_T1']

In [ ]:
# def normalize(arr):
#     return (arr - arr.min()) / (arr.max() - arr.min() + 1e-6)

# def compute_cloud_mask(chunk):
#     # Extract relevant bands
#     blue = chunk[1]
#     red = chunk[3]
#     nir = chunk[4]
#     swir1 = chunk[5]
#     thermal = chunk[9]

#     # Normalized indices
#     ndvi = (nir - red) / (nir + red + 1e-6)
    
#     blue_n = normalize(blue)
#     swir1_n = normalize(swir1)

#     # Cloud condition
#     potential_cloud = (
#         (blue_n > 0.2) &
#         (swir1_n > 0.2) &
#         (ndvi < 0.3) &
#         (thermal < 30000)  
#     )

#     return potential_cloud.astype(np.uint8)

In [ ]:
# def process_block(block):
#     mask = compute_cloud_mask(block)
#     return np.array([[mask.mean()]], dtype='float32') 

#     # Apply over chunks using map_blocks
# cloud_fraction = arr.data.map_blocks(
#     lambda block: process_block(block),
#     dtype='float32',
#     drop_axis=0,  # drop 'band' axis
# )

In [ ]:
# # print(cloud_fraction)
# cloud_cover_da = xr.DataArray(
#     cloud_fraction,
#     dims=['chunk_y', 'chunk_x'],
#     name='cloud_fraction'
# )
# print(cloud_cover_da)

In [ ]:
mean = cloud_cover_da.mean(dim=['chunk_y', 'chunk_x']).compute()
variance = cloud_cover_da.var().compute()

print(f"Mean cloud cover: {mean.item() * 100:.2f}%")
print(f"Variance of cloud cover: {(variance.item() * 100**2):.2f} (% squared)")

In [ ]:
def simple_cloud_detection_bands_v2(
    scene_da: xr.DataArray,
    blue_band_selector: Union[str, int],
    nir_band_selector: Union[str, int],
    # Provide the order of band names as they exist in your Zarr file's 'band' dimension
    # This is only needed if you pass string selectors (e.g., 'blue', 'nir')
    band_names_in_data: Optional[List[str]] = None,
    red_band_selector: Optional[Union[str, int]] = None,
    blue_reflectance_threshold: float = 0.25,
    nir_reflectance_threshold: float = 0.20,
    epsilon: float = 1e-8
) -> xr.DataArray:
    """
    Cloud detection operating on selected bands, robust to Zarr files
    that only have dimension names but not band coordinate labels.
    """
    print(f"--- Inside simple_cloud_detection_bands_v2 ---")
    print(f"Input scene_da: {scene_da.name}, dims: {scene_da.dims}, shape: {scene_da.shape}, chunks: {scene_da.chunks}")
    if 'band' not in scene_da.dims:
        raise ValueError(f"Input DataArray must have a 'band' dimension. Found dims: {scene_da.dims}")

    # Helper to get band index
    def get_band_index(selector: Union[str, int], dim_name: str = 'band') -> int:
        if isinstance(selector, int):
            return selector
        elif isinstance(selector, str):
            if band_names_in_data is None:
                raise ValueError(
                    "band_names_in_data must be provided if band selectors are strings "
                    "and the DataArray does not have named band coordinates."
                )
            try:
                return band_names_in_data.index(selector)
            except ValueError:
                raise ValueError(
                    f"Band name '{selector}' not found in band_names_in_data: {band_names_in_data}"
                )
        else:
            raise TypeError(f"Band selector must be str or int, got {type(selector)}")

    # --- 1. Select Bands using integer positions ---
    try:
        blue_idx = get_band_index(blue_band_selector)
        blue_band = scene_da.isel(band=blue_idx)
        print(f"Selected Blue band (selector: '{blue_band_selector}', index: {blue_idx})")

        nir_idx = get_band_index(nir_band_selector)
        nir_band = scene_da.isel(band=nir_idx)
        print(f"Selected NIR band (selector: '{nir_band_selector}', index: {nir_idx})")

        red_band = None
        if red_band_selector is not None:
            red_idx = get_band_index(red_band_selector)
            red_band = scene_da.isel(band=red_idx)
            print(f"Selected Red band (selector: '{red_band_selector}', index: {red_idx})")

    except IndexError as e:
        print(f"Error during band selection: {e}")
        print(f"Max index for 'band' dimension (size-1): {scene_da.sizes.get('band', 0) - 1}")
        print(f"Attempted blue_idx: {blue_idx if 'blue_idx' in locals() else 'N/A'}, "
              f"nir_idx: {nir_idx if 'nir_idx' in locals() else 'N/A'}")
        raise

    # --- 2. Placeholder Cloud Logic ---
    print(f"Applying blue_reflectance_threshold: {blue_reflectance_threshold}")
    print(f"Applying nir_reflectance_threshold: {nir_reflectance_threshold}")

    potential_clouds = (blue_band > blue_reflectance_threshold) & \
                       (nir_band > nir_reflectance_threshold)

    potential_clouds = potential_clouds.rename("cloud_mask_placeholder")
    # ... (add attributes as before)
    print(f"--- Exiting simple_cloud_detection_bands_v2 ---")
    return potential_clouds

In [ ]:
ORDERED_BAND_NAMES_IN_FILE = ['blue_channel', 'green_channel', 'near_infrared', 'red_channel',]

In [ ]:
print("\nCalling simple_cloud_detection_bands_v2...")
cloud_mask_da = simple_cloud_detection_bands_v2(
    scene_da=data_array_to_process,
    blue_band_selector='blue_channel',    # String name
    nir_band_selector='near_infrared', # String name
    band_names_in_data=ORDERED_BAND_NAMES_IN_FILE, # Crucial mapping
    blue_reflectance_threshold=0.25,
    nir_reflectance_threshold=0.20
)
print(f"\nOutput Cloud Mask DataArray (cloud_mask_da) (lazy Dask array):")
print(cloud_mask_da)

In [ ]:
print("\nCalculating cloud cover percentage (triggers Dask computation)...")
# Ensure we only average over spatial dimensions if the mask is already 2D
if len(cloud_mask_da.dims) == 2 and 'y' in cloud_mask_da.dims and 'x' in cloud_mask_da.dims:
    cloud_proportion = cloud_mask_da.mean(dim=['y', 'x'])
else: # A more general mean if dimensions are different, though not expected here
    cloud_proportion = cloud_mask_da.mean()
cloud_percentage = cloud_proportion.compute() * 100.0
print(f"\nEstimated Cloud Cover Percentage: {cloud_percentage:.2f}%")

In [ ]:
data_variable_name = 'LC08_L1TP_032037_20160420_20170223_01_T1'
data_array_to_process = ds_lazy[data_variable_name]

# 2. Define your cloud posterior function (example from before)
# This function expects a 1D numpy array of spectral values for a single pixel
def simple_cloud_posterior(pixel_spectral_bands, bright_threshold=0.6, whiteness_factor=0.15):
    """
    Calculate a simple cloud posterior probability for a single pixel.
    Args:
        pixel_spectral_bands (np.ndarray): 1D array of spectral values (e.g., [R, G, B]).
                                          Assumed to be scaled roughly 0-1.
        bright_threshold (float): Pixels with mean reflectance below this are considered clear.
        whiteness_factor (float): How "white" a pixel needs to be.
    Returns:
        float: A pseudo-posterior probability (0.0 to 1.0) of the pixel being cloud.
    """
    if not isinstance(pixel_spectral_bands, np.ndarray): # Should be numpy array from dask chunk
        pixel_spectral_bands = np.array(pixel_spectral_bands)

    mean_reflectance = np.mean(pixel_spectral_bands)

    if mean_reflectance < bright_threshold:
        return 0.0  # Definitely not cloud if not bright enough

    if mean_reflectance > 1e-6:
        color_variation = np.std(pixel_spectral_bands) / mean_reflectance
    else:
        color_variation = 1.0

    brightness_score = min(1.0, max(0.0, (mean_reflectance - bright_threshold) / (1.0 - bright_threshold + 1e-6)))
    whiteness_score = max(0.0, 1.0 - (color_variation / whiteness_factor))
    cloud_prob = brightness_score * whiteness_score
    return float(cloud_prob)


# 3. Apply the function chunkwise using xarray.apply_ufunc
cloud_posterior_da = xr.apply_ufunc(
    simple_cloud_posterior,         # The function to apply
    data_array_to_process,          # The input DataArray (NOT the Dataset)
    input_core_dims=[['x']],        # The dimension your function operates on is 'x'
    output_core_dims=[[]],          # Your function returns a scalar
    exclude_dims=set(('x',)),       # The 'x' dimension is consumed by the function
    dask="parallelized",            # Enable Dask parallelization
    output_dtypes=[float],          # The data type of the output
    kwargs={'bright_threshold': 0.65, 'whiteness_factor': 0.2} # Pass additional args
)

# The resulting cloud_posterior_da will have dimensions ('band', 'y')
# because the 'x' dimension was consumed.
cloud_posterior_da.name = 'cloud_posterior' # Give it a name

print("Input DataArray to process:")
print(data_array_to_process)
print("\nCloud Posterior DataArray structure:")
print(cloud_posterior_da)
print("\nCloud Posterior DataArray Chunks:")
print(cloud_posterior_da.chunks)

In [ ]:
print("--- Inspecting cloud_posterior_da ---")
print(repr(cloud_posterior_da)) # Full representation
print("\nShape:", cloud_posterior_da.shape)
print("Dimensions:", cloud_posterior_da.dims)
print("Chunks:", cloud_posterior_da.chunks)
print("Dtype:", cloud_posterior_da.dtype)

# Important: Inspect the _meta attribute of the underlying Dask array
if hasattr(cloud_posterior_da, 'data') and hasattr(cloud_posterior_da.data, '_meta'):
    print("Dask array _meta:", repr(cloud_posterior_da.data._meta))
else:
    print("Could not access Dask array _meta.")

# Try to compute a small part to see the actual values in a chunk
print("\nComputing a small slice of cloud_posterior_da")

In [ ]:
# cloud_posterior_values = cloud_posterior_da.values

# These are still Dask arrays. Compute() triggers calculation.
mean_cloud_cover = cloud_posterior_da.mean(skipna=True)
variance_cloud_cover = cloud_posterior_da.var(skipna=True)

print("Computing Cloud mean...")
cloud_mean_computed = mean_cloud_cover.compute()
print(f"Computed cloud Mean: {cloud_mean_computed.item()}") # .item() to get Python scalar

print("Computing cloud variance...")
cloud_variance_computed = variance_cloud_cover.compute()
print(f"Computed cloud Variance: {cloud_variance_computed.item()}")


In [ ]:
RED_BAND_INDEX_IN_X_DIM = 0
NIR_BAND_INDEX_IN_X_DIM = 1
DATA_VARIABLE_NAME = 'LC08_L1TP_050024_20160520_20170324_01_T1'

In [ ]:
multi_band_da = ds_lazy[DATA_VARIABLE_NAME]
print(f"Extracted multi-band DataArray ('{multi_band_da.name}'): \n{multi_band_da}")

In [ ]:
print(f"Selecting Red band using index {RED_BAND_INDEX_IN_X_DIM} along 'x' dimension...")
# .isel(x=...) selects by integer position along the 'x' dimension.
red_da = multi_band_da.isel(x=RED_BAND_INDEX_IN_X_DIM)
# Rename dimensions for clarity if needed, e.g., if 'band' is height and 'y' is width
# red_da = red_da.rename({'band': 'height', 'y': 'width'})


print(f"Selecting NIR band using index {NIR_BAND_INDEX_IN_X_DIM} along 'x' dimension...")
nir_da = multi_band_da.isel(x=NIR_BAND_INDEX_IN_X_DIM)
# nir_da = nir_da.rename({'band': 'height', 'y': 'width'}) # Keep dimensions consistent with red_da

print(f"Red band selected (lazy): \n{red_da}")
print(f"NIR band selected (lazy): \n{nir_da}")

# --- 3. Calculate NDVI (Chunk-wise operation) ---
epsilon = 1e-8
print("Calculating NDVI (lazy Dask computation)...")
ndvi_da = (nir_da - red_da) / (nir_da + red_da + epsilon)
ndvi_da = ndvi_da.rename("ndvi")

ndvi_da.attrs.update({
    'long_name': 'Normalized Difference Vegetation Index',
    'units': '1',
    'formula': '(NIR - Red) / (NIR + Red)',
    'source_data_variable': DATA_VARIABLE_NAME,
    'red_band_source_index_in_x_dim': RED_BAND_INDEX_IN_X_DIM,
    'nir_band_source_index_in_x_dim': NIR_BAND_INDEX_IN_X_DIM,
    'epsilon_for_division': epsilon
})

print(f"NDVI DataArray (lazy): \n{ndvi_da}")
print(f"NDVI Chunks: {ndvi_da.chunks}")


In [ ]:
print("Calculating NDVI mean and variance (lazy Dask computation)...")

# Calculate mean, explicitly skipping NaNs if they might exist
# If your data is clean (no NaNs expected), skipna=False might be slightly faster.
ndvi_mean_lazy = ndvi_da.mean(skipna=True)
ndvi_variance_lazy = ndvi_da.var(skipna=True)

# We need to compute these values to put them in the JSON.
# This will trigger reading the necessary data for the whole ndvi_da array.
# If ndvi_da is very large, computing global mean/variance can be memory intensive
# if not chunked optimally for reduction. Dask handles this reasonably well.
print("Computing NDVI mean...")
ndvi_mean_computed = ndvi_mean_lazy.compute()
print(f"Computed NDVI Mean: {ndvi_mean_computed.item()}") # .item() to get Python scalar

print("Computing NDVI variance...")
ndvi_variance_computed = ndvi_variance_lazy.compute()
print(f"Computed NDVI Variance: {ndvi_variance_computed.item()}")

In [ ]:
input_zarr_name = DATA_VARIABLE_NAME # Using the main variable name
output_s3_bucket = zarr_catalog_url.split('/')[2]
output_zarr_prefix = "posterior"
output_ndvi_zarr_name = f"{input_zarr_name}_ndvi.zarr" # Might want a cleaner name
output_ndvi_s3_path = f"s3://{output_s3_bucket}/{output_zarr_prefix}/{output_ndvi_zarr_name}"
print(f"Output NDVI Zarr will be saved to: {output_ndvi_s3_path}")

output_s3_map_root = f"{output_s3_bucket}/{output_zarr_prefix}/{output_ndvi_zarr_name}"
if 's3' not in locals() or not isinstance(s3, s3fs.S3FileSystem):
    s3 = s3fs.S3FileSystem(**s3_options)
s3_map_for_writing = s3fs.S3Map(root=output_s3_map_root, s3=s3, check=False)

# --- 5. Save the NDVI DataArray as a new Zarr store (Chunk-wise write) ---
ndvi_ds_to_save = ndvi_da.to_dataset()
print("Saving NDVI Zarr to S3...")
try:
    task = ndvi_ds_to_save.to_zarr(
        store=s3_map_for_writing,
        mode='w',
        consolidated=True,
        compute=True
    )
    print(f"NDVI Zarr store successfully written to {output_ndvi_s3_path}")
except Exception as e:
    print(f"Error saving NDVI Zarr to S3: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\nNDVI calculation and Zarr upload complete.")

In [ ]:
scene_id_for_summary = input_zarr_name

summary_data = {
    "scene_id": scene_id_for_summary,
    "processed_product": "NDVI",
    "source_zarr": zarr_catalog_url, # Link back to the input
    "output_ndvi_zarr": output_ndvi_s3_path,
    "mean": float(ndvi_mean_computed.item()), # Ensure it's a standard float
    "variance": float(ndvi_variance_computed.item()),
    "calculation_timestamp_utc": datetime.now(timezone.utc).isoformat()
}

summary_json_content = json.dumps(summary_data, indent=2)
print(f"Summary JSON content: \n{summary_json_content}")

summary_json_s3_path = f"s3://{output_s3_bucket}/posterior/ndvi_summary_{scene_id_for_summary}.json"


try:
    print(f"Uploading summary JSON to {summary_json_s3_path}...")
    with s3.open(summary_json_s3_path, 'wb') as f: # s3.open expects path within bucket
        f.write(summary_json_content.encode('utf-8'))
    print("Summary JSON uploaded successfully.")
except Exception as e:
    print(f"Error uploading summary JSON: {e}")

In [ ]:
fs_for_ledger = s3

ledger_bucket = output_s3_bucket
ledger_file_name = "ledgers.csv"
ledger_s3_key = f"{ledger_file_name}"
ledger_full_s3_path = f"s3://{ledger_bucket}/{ledger_s3_key}"

print(f"Updating placeholder ledger at: {ledger_full_s3_path}")

try:
    summary_json_bytes = summary_json_content.encode('utf-8')
    sha256_hash = hashlib.sha256(summary_json_bytes).hexdigest()
    timestamp_utc_str = summary_data["calculation_timestamp_utc"] # Use timestamp from summary

    scene_id_ledger = summary_data["scene_id"]
    mean_val_ledger = summary_data["mean"]
    var_val_ledger = summary_data["variance"]
    stage_ledger = "NDVI_Posterior_Placeholder"

    new_ledger_line = f"{scene_id_ledger},{sha256_hash},{timestamp_utc_str},{stage_ledger},{mean_val_ledger},{var_val_ledger}\n"

    if fs_for_ledger.exists(ledger_s3_key):
        print(f"Ledger file {ledger_s3_key} exists, appending.")
        with fs_for_ledger.open(ledger_s3_key, "ab") as f:
            f.write(new_ledger_line.encode("utf-8"))
    else:
        print(f"Ledger file {ledger_s3_key} does not exist, creating with header.")
        header = "scene_id,summary_json_sha256,timestamp_utc,stage,mean,variance\n"
        content = header + new_ledger_line
        with fs_for_ledger.open(ledger_full_s3_path, "wb") as f:
            f.write(content.encode("utf-8"))
    print(f"Ledger updated successfully for placeholder scene {scene_id_ledger}.")

except Exception as e:
    print(f"Failed to update ledger for placeholder scene {scene_id_for_summary}: {e}")
    import traceback
    traceback.print_exc()

print("\nPlaceholder summary JSON and ledger update complete.")